In [1]:
import django, sys, os
sys.path.append('/home/max/software/django-tmv/tmv_mcc-apsis/BasicBrowser')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *
from tmv_app.models import *
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from itertools import product, combinations
from utils.text import *
from pycountry_convert import  country_alpha2_to_continent_code, country_alpha3_to_country_alpha2
import scipy.stats as st
import pandas as pd

In [2]:
df = pd.read_csv('IdLabelPrediction.csv')

included = df[(df['prediction']>0.35) | (df['label']==1)]
len(included.id.unique())

df.head()

,id,label,prediction
0,3546488,NaN,0.018036
1,2399733,NaN,0.138054
2,730310,NaN,0.188601
3,3371201,NaN,0.020143
4,3369993,NaN,0.009315


In [3]:
doc_df = pd.DataFrame.from_dict(
    Doc.objects.filter(id__in=included.id).values('id','title','content','PY','wc__oecd','wosarticle__di')
)

print(doc_df.shape)

doc_df.head()

(11111, 6)


,id,title,content,PY,wc__oecd,wosarticle__di
0,3255833,Assessment of spatiotemporal variability of re...,Reference evapotranspiration (ETo) is a key co...,2019.0,None,10.1016/j.agwat.2018.09.037
1,702492,The Importance of Bank Vole Density and Rainy ...,"Pathogenic hantaviruses (family Bunyaviridae, ...",2014.0,Natural Sciences,10.1371/journal.pone.0111663
2,14865,Summertime Acute Heat Illness in U.S. Emergenc...,BACKGROUND: Patients with acute heat illness p...,2014.0,Natural Sciences,10.1289/ehp.1306796
3,3457216,Health-Related Quality of Life of Chinese Eart...,As one of the most destructive natural disaste...,2014.0,None,10.1007/s11205-013-0525-2
4,282602,Health coping strategies of the people vulnera...,Background: Among the many challenges faced by...,2013.0,Medical and Health Sciences,10.1186/1471-2458-13-565


In [4]:
doc_df['DOI'] = "["+ doc_df['wosarticle__di'] + "](https://dx.doi.org/" + doc_df['wosarticle__di'] + ")"
doc_df.head()

,id,title,content,PY,wc__oecd,wosarticle__di,DOI
0,3255833,Assessment of spatiotemporal variability of re...,Reference evapotranspiration (ETo) is a key co...,2019.0,None,10.1016/j.agwat.2018.09.037,[10.1016/j.agwat.2018.09.037](https://dx.doi.o...
1,702492,The Importance of Bank Vole Density and Rainy ...,"Pathogenic hantaviruses (family Bunyaviridae, ...",2014.0,Natural Sciences,10.1371/journal.pone.0111663,[10.1371/journal.pone.0111663](https://dx.doi....
2,14865,Summertime Acute Heat Illness in U.S. Emergenc...,BACKGROUND: Patients with acute heat illness p...,2014.0,Natural Sciences,10.1289/ehp.1306796,[10.1289/ehp.1306796](https://dx.doi.org/10.12...
3,3457216,Health-Related Quality of Life of Chinese Eart...,As one of the most destructive natural disaste...,2014.0,None,10.1007/s11205-013-0525-2,[10.1007/s11205-013-0525-2](https://dx.doi.org...
4,282602,Health coping strategies of the people vulnera...,Background: Among the many challenges faced by...,2013.0,Medical and Health Sciences,10.1186/1471-2458-13-565,[10.1186/1471-2458-13-565](https://dx.doi.org/...


In [5]:
geos = pd.read_csv('data/geo_df.csv')
geos = geos.drop_duplicates(["doc_id","geonameid"])

geos = geos[geos['country_conf']>0.85]

geos.lat = geos.lat.astype(float)
geos.lon = geos.lon.astype(float)

geos.loc[geos["word"]=="Pakistan","feature_code"]="PCLI"
geos.loc[geos["word"]=="Colombia","feature_code"]="PCLI"
geos.loc[geos["word"]=="Argentina","feature_code"]="PCLI"

geos = geos[geos["word"]!="MMT"]

import pycountry_convert as pc
def get_cont(x):
    continents = {
        'NA': 'North America',
        'SA': 'South America', 
        'AS': 'Asia',
        'OC': 'Oceania',
        'AF': 'Africa',
        'EU': 'Europe'
    }
    try:
        return continents[pc.country_alpha2_to_continent_code(pc.country_alpha3_to_country_alpha2(x['country_predicted']))]
    except:
        return None
geos['continent'] = geos.apply(lambda x: get_cont(x), axis=1)
other_country_continents = [
    ("ESH","Africa"),
    ("TLS","Oceania"),
    ("XKX","Europe")
    
]
for country,continent in other_country_continents:
    geos.loc[geos['country_predicted']==country,"continent"] = continent

geos.head()

,admin1,lat,lon,country_code3,geonameid,place_name,feature_class,feature_code,doc_id,word,country_predicted,country_conf,continent
0,Shandong,36.66833,116.99722,CHN,1805753,Jinan,P,PPLA,1061,Jinan,CHN,0.999811,Asia
2,NaN,35.00000,105.00000,CHN,1814991,People’s Republic of China,A,PCLI,1061,China,CHN,0.999764,Asia
5,Manicaland,-19.00000,32.41667,ZWE,887358,Manicaland Province,A,ADM1,5923,Manicaland,ZWE,0.937507,Africa
6,Mashonaland East,-17.96667,31.66667,ZWE,886842,Mashonaland East Province,A,ADM1,5923,Mashonaland,ZWE,0.937507,Africa
7,NaN,39.69450,-8.13057,PRT,2264397,Portuguese Republic,A,PCLI,7057,Portugal,PRT,0.999579,Europe


In [6]:
df_places = doc_df.merge(geos,left_on="id",right_on="doc_id")
df_places.to_csv('../DIFID/data/df_places.csv',index=False)
df_places.to_csv('../dash-dfid/data/df_places.csv',index=False)
df_places.to_csv('data/df_places.csv',index=False)
df_places.head()

,id,title,content,PY,wc__oecd,wosarticle__di,DOI,admin1,lat,lon,country_code3,geonameid,place_name,feature_class,feature_code,doc_id,word,country_predicted,country_conf,continent
0,3255833,Assessment of spatiotemporal variability of re...,Reference evapotranspiration (ETo) is a key co...,2019.0,None,10.1016/j.agwat.2018.09.037,[10.1016/j.agwat.2018.09.037](https://dx.doi.o...,NaN,35.00000,105.00000,CHN,1814991,People’s Republic of China,A,PCLI,3255833,China,CHN,1.000000,Asia
1,3255833,Assessment of spatiotemporal variability of re...,Reference evapotranspiration (ETo) is a key co...,2019.0,None,10.1016/j.agwat.2018.09.037,[10.1016/j.agwat.2018.09.037](https://dx.doi.o...,NaN,34.00000,116.00000,CHN,1807463,Huanghuai Pingyuan,T,PLN,3255833,North China,CHN,0.999492,Asia
2,702492,The Importance of Bank Vole Density and Rainy ...,"Pathogenic hantaviruses (family Bunyaviridae, ...",2014.0,Natural Sciences,10.1371/journal.pone.0111663,[10.1371/journal.pone.0111663](https://dx.doi....,Southern Savonia,61.52728,28.17495,FIN,640504,Puumala,P,PPLA3,702492,Puumala,FIN,0.983592,Europe
3,702492,The Importance of Bank Vole Density and Rainy ...,"Pathogenic hantaviruses (family Bunyaviridae, ...",2014.0,Natural Sciences,10.1371/journal.pone.0111663,[10.1371/journal.pone.0111663](https://dx.doi....,NaN,62.00000,15.00000,SWE,2661886,Kingdom of Sweden,A,PCLI,702492,Sweden,SWE,0.999981,Europe
4,3457216,Health-Related Quality of Life of Chinese Eart...,As one of the most destructive natural disaste...,2014.0,None,10.1007/s11205-013-0525-2,[10.1007/s11205-013-0525-2](https://dx.doi.org...,Sichuan,30.50000,102.66667,CHN,1794299,Sichuan Sheng,A,ADM1,3457216,Sichuan,CHN,0.997501,Asia


In [7]:
df = df_places
df["content"].str.wrap(70).str.replace("\n","<br>",regex=False)[0]

'Reference evapotranspiration (ETo) is a key component of the<br>hydrological cycle, and it plays a vital role in agricultural, forest,<br>and environmental management. This study assesses the capability of<br>hot spot geospatial analysis to determine statistically significant<br>spatial clusters of high and low ETo. in China over the period of<br>1970-2014, based on the daily data from 598 weather stations. The<br>global controlling factors affecting ETo across continental China are<br>investigated using global ordinary least square regression (OLS)<br>model. The spatial relationship between ETo and climatic variables is<br>explored using local geographic weighted regression (GWR) model. It<br>was found that for China as a whole, ETo decreased significantly from<br>1970 to 1993 at a rate of 14.91 mm decade(-1), while the trend began<br>increasing by 16.50 mm decade(-1) from 1993 to 2014. The hot spot<br>analysis showed that the regional distribution of statistically<br>significant spa

In [8]:
#stock = df_places[df_places["word"]=="Stockholm"].id
#df_places[(df_places.id.isin(stock)) & (df_places.country_predicted!="SWE")]

In [11]:
doc_continent = geos[pd.notna(geos["continent"])][["doc_id","continent"]].groupby('doc_id').agg(lambda x: st.mode(x)[0]).reset_index()
doc_continent.head()


,doc_id,continent
0,1061,Asia
1,5923,Africa
2,7057,Europe
3,7896,North America
4,10055,Asia


In [12]:
doc_df = pd.merge(doc_df,doc_continent,how="left",left_on="id",right_on="doc_id")
doc_df = doc_df.drop("doc_id",axis=1)
doc_df.head()

,id,title,content,PY,wc__oecd,wosarticle__di,DOI,continent
0,3255833,Assessment of spatiotemporal variability of re...,Reference evapotranspiration (ETo) is a key co...,2019.0,None,10.1016/j.agwat.2018.09.037,[10.1016/j.agwat.2018.09.037](https://dx.doi.o...,Asia
1,702492,The Importance of Bank Vole Density and Rainy ...,"Pathogenic hantaviruses (family Bunyaviridae, ...",2014.0,Natural Sciences,10.1371/journal.pone.0111663,[10.1371/journal.pone.0111663](https://dx.doi....,Europe
2,14865,Summertime Acute Heat Illness in U.S. Emergenc...,BACKGROUND: Patients with acute heat illness p...,2014.0,Natural Sciences,10.1289/ehp.1306796,[10.1289/ehp.1306796](https://dx.doi.org/10.12...,NaN
3,3457216,Health-Related Quality of Life of Chinese Eart...,As one of the most destructive natural disaste...,2014.0,None,10.1007/s11205-013-0525-2,[10.1007/s11205-013-0525-2](https://dx.doi.org...,Asia
4,282602,Health coping strategies of the people vulnera...,Background: Among the many challenges faced by...,2013.0,Medical and Health Sciences,10.1186/1471-2458-13-565,[10.1186/1471-2458-13-565](https://dx.doi.org/...,Asia


In [13]:
doc_df.continent.unique()

array(['Asia', 'Europe', nan, 'Oceania', 'North America', 'South America',
       'Africa'], dtype=object)

In [14]:
doc_df.to_csv('../DIFID/data/doc_information.csv',index=False)
doc_df.to_csv('data/doc_information.csv',index=False)

In [15]:
django.db.connection.close()
run_id = 2914
stat = RunStats.objects.get(pk=run_id)
q = stat.query

# position the papers in a 2-dimensional topic space, with perplexity p
# unless you pass the argument force_overwrite=True, the results are not recalculated
s = 0
p = 50
tsne_results, r_ind = stat.calculate_tsne('tsne_results/data', p)

We've already calculated the required matrices!
We've already calculated the tsne positions


In [16]:
tsne_df = pd.DataFrame(tsne_results)
tsne_df.columns = ["tsne-1","tsne-2"]
tsne_df['doc_id'] = r_ind
print(tsne_df.shape)
tsne_df.to_csv('../DIFID/data/tsne_results.csv',index=False)
tsne_df.head()

(16964, 3)


,tsne-1,tsne-2,doc_id
0,-11.650991,-11.426455,1474561
1,-15.316335,5.142924,3440643
2,-2.607396,-19.999853,3538950
3,-7.297103,-7.294213,1474569
4,-0.096713,23.515265,2097161


In [17]:
dts = pd.DataFrame.from_dict(
    DocTopic.objects.filter(run_id=run_id).values('doc_id','topic_id','score')
)
dts.to_csv('../DIFID/data/doctopic.csv',index=False)
dts.to_csv('../dash-dfid/data/doctopic.csv',index=False)
dts.to_csv('data/doctopic.csv',index=False)

dts.pivot(index="doc_id",columns="topic_id",values="score").fillna(0).to_csv('../DIFID/data/doctopic_wide.csv')

In [18]:
django.db.connection.close()
topic_df = pd.DataFrame.from_dict(
    Topic.objects.filter(run_id=run_id).values('id','title','top_words')
)
topic_df["short_title"] = topic_df["title"].str.split(",").str[0]
topic_df.to_csv('data/topic_info.csv',index=False)
topic_df.to_csv('../DIFID/data/topic_info.csv',index=False)
topic_df.to_csv('../dash-dfid/data/topic_info.csv',index=False)
topic_df.head()

,id,title,top_words,short_title
0,212274,"Infection, Infectious disease","[diseas, infecti, respiratori, cardiovascular,...",Infection
1,212321,"Mental, Mental health","[mental, health, disord, physic, psycholog, di...",Mental
2,212322,"Drought, Drought","[drought, impact, index, veget, sever, agricul...",Drought
3,212323,"Suicide, Suicide","[suicid, associ, rate, attempt, male, men, stu...",Suicide
4,212327,"Daily, Daily data & trends","[day, daili, period, dure, visit, number, incr...",Daily
